<a href="https://colab.research.google.com/github/TirendazAcademy/PyTorch-Lightning-Tutorials/blob/main/CNN_with_Lightning_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightning -q 

In [2]:
import os
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt
import torchvision

In [3]:
class MnistDataModule(pl.LightningDataModule):
    def __init__(self):
        #this is the init function where we will defnine the architecture
        super().__init__()

    def prepare_data(self):     
        # This contains the manupulation on data that needs to be done only once such as downloading it

        #download the MNIST dataset
        MNIST(os.getcwd(), train=True, download =True)
        MNIST(os.getcwd(), train=False, download =True)
        
        # See here I have set download to false as it is already downloaded
        mnist_train=MNIST(os.getcwd(), train=True, download =False,transform=transforms.ToTensor())
  
        #dividing into validation and training set
        self.train_set, self.val_set= random_split(mnist_train,[55000,5000])
    
    def train_dataloader(self):
        # REQUIRED
        # This is an essential function. Needs to be included in the code              
        return DataLoader(self.train_set, batch_size=128, num_workers=32)
        
    def val_dataloader(self):
        # OPTIONAL
        #loading validation dataset
        return DataLoader(self.val_set, batch_size=128,num_workers=32)

    def test_dataloader(self):
        # OPTIONAL
        # loading test dataset
        return DataLoader(MNIST(os.getcwd(), train=False, download=False, transform=transforms.ToTensor()), batch_size=128,num_workers=32)

In [4]:
#defining the model
class NN(pl.LightningModule):
    def __init__(self):
        #this is the init function where we will defnine the architecture
        super().__init__()
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=0.5))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=0.5))
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            torch.nn.Dropout(p=0.5))

        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.5))
        self.dense1_bn = torch.nn.BatchNorm1d(625)
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        self.dense2_bn = torch.nn.BatchNorm1d(10)
        torch.nn.init.xavier_uniform_(self.fc2.weight) # initialize parameters

    def forward(self,x):
        # evaluating the batch data as it moves forward in the netowrk
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        # out = self.fc1(out)
        out = self.dense1_bn(self.fc1(out))
        
        # out = self.fc2(out)
        out = self.dense2_bn(self.fc2(out))
        # print(out.shape)
        return F.softmax(out,dim=1)
    
    def training_step(self,batch,batch_idx):
        #REQUIRED
        # extracting input and output from the batch
        x, labels=batch
        pred=self.forward(x)

        correct=pred.argmax(dim=1).eq(labels).sum().item()
        total=len(labels)
        #calculating the loss
        train_loss = F.cross_entropy(pred, labels)

        #logs
        logs={"train_loss": train_loss}

        output={
            #REQUIRED: It ie required for us to return "loss"
            "loss": train_loss,
            #optional for logging purposes
            "log": logs,
            "correct": correct,
            "total": total
        }
        return output

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        # The code that runs as we forward pass a validation batch
        x, y = batch
        y_hat = self(x)
        correct=y_hat.argmax(dim=1).eq(y).sum().item()
        total=len(y)
        return {'val_loss': F.cross_entropy(y_hat, y),"correct": correct,"total": total}

    def test_step(self, batch, batch_nb):
        # OPTIONAL
        # The code that runs as we forward pass a test batch

        x, y = batch
        y_hat = self(x)
        correct=y_hat.argmax(dim=1).eq(y).sum().item()
        total=len(y)
        # returning the batch_dictionary
        return {'test_loss': F.cross_entropy(y_hat, y),
                "correct": correct,
                "total": total}

    def configure_optimizers(self):
        # REQUIRED
        # Can return multiple optimizers and learning_rate schedulers
        return torch.optim.Adam(self.parameters(), lr=0.0005)

In [5]:
dm = MnistDataModule()
model = NN()
trainer = pl.Trainer(fast_dev_run=True, accelerator="auto", devices="auto")
trainer.fit(model, dm)

<ipython-input-4-ab1896426550>:24: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc1.weight)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


100%|██████████| 9912422/9912422 [00:00<00:00, 87262901.35it/s]


Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 25680664.37it/s]


Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw



...

Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw



INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type        | Params
------------------------------------------
0 | layer1    | Sequential  | 320   
1 | layer2    | Sequential  | 18.5 K
2 | layer3    | Sequential  | 73.9 K
3 | fc1       | Linear      | 1.3 M 
4 | layer4    | Sequential  | 1.3 M 
5 | dense1_bn | BatchNorm1d | 1.2 K 
6 | fc2       | Linear      | 6.3 K 
7 | dense2_bn | BatchNorm1d | 20    
------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.523     Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation migh

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1` reached.


In [6]:
trainer = pl.Trainer(max_epochs=10, accelerator="auto", devices="auto")
trainer.fit(model, dm)
trainer.validate(model, dm)
trainer.test(model, dm)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type        | Params
------------------------------------------
0 | layer1    | Sequential  | 320   
1 | layer2    | Sequential  | 18.5 K
2 | layer3    | Sequential  | 73.9 K
3 | fc1       | Linear      | 1.3 M 
4 | layer4    | Sequential  | 1.3 M 
5 | dense1_bn | BatchNorm1d | 1.2 K 
6 | fc2       | Linear      | 6.3 K 
7 | dense2_bn | BatchNorm1d | 20    
------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.523     Total estimated model params size (

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{}]

# Resource

- [TensorBoard with PyTorch Lightning](https://learnopencv.com/tensorboard-with-pytorch-lightning/)


🔗 Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Instagram](https://www.instagram.com/tirendazacademy) | [GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎